In [1]:
import pandas as pd

from fluxo.dados import Dados
from fluxo.carga_dados import CargaDados
from fluxo.limpeza_dados import LimpezaDados
from fluxo.preprocessamento import Preprocessamento
from fluxo.extracao_caracteristicas import ExtracaoCaracteristicas
from fluxo.treinamento import Treinamento
from fluxo.classificador import Classificador

from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
pd.set_option('max_colwidth', 180)

## Carga de Dados

In [3]:
dados = Dados()

In [4]:
cg = CargaDados()
cg.executar(dados)

INFO:root:Carregando dados de segmentos...
INFO:root:206488 registros carregados.


## Limpeza de Dados

In [5]:
lp = LimpezaDados(dados)
lp.executar(dados)

INFO:root:Excluindo segmentos dos atos que não são ADE...
INFO:root:91448 segmentos de atos não ADE excluídos.
INFO:root:Restaram 115040 segmentos de atos ADE.
INFO:root:Removendo segmentos não representativos...
INFO:root:6827 segmentos não representativos excluídos.
INFO:root:Restaram 108213 segmentos representativos.
INFO:root:Removendo segmentos nulos...
INFO:root:69 segmentos nulos excluídos.
INFO:root:Restaram 108144 segmentos não nulos.
INFO:root:Removendo tags HTML...
INFO:root:Removendo caracteres de escape HTML...
INFO:root:Reclassificando segmentos não identificados...
INFO:root:1418 segmentos reclassificados como artigos.
INFO:root:Limpeza de dados concluída.


## Preprocessamento

In [6]:
pp = Preprocessamento()
pp.executar(dados)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/91872936504/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:root:Convertendo caracteres para minúsculo...
INFO:root:Removendo pontuação...
INFO:root:Realizando tokenização...
INFO:root:Removendo stopwords...
INFO:root:Removendo tokens menores que 2 caracteres...
INFO:root:Reconstruindo texto a partir dos tokens...
INFO:root:Preprocessamento concluído.


## Extração de Características

In [12]:
ec = ExtracaoCaracteristicas(dados)
ec.executar(dados)

INFO:root:Executando vetorização TF-IDF...
INFO:root:Tamanho do vocabulario: 31988 termos
INFO:root:Extração de características concluída.


## Treinamento e Teste

In [8]:
t = Treinamento()

In [14]:
# Linear SVC (SVM)
estimador = OneVsOneClassifier(LinearSVC())
t.adicionar_modelo(Classificador('LinearSVC-OneVsOne', estimador))
t.treinar_modelos(dados)

estimador = OneVsRestClassifier(LinearSVC())
t.adicionar_modelo(Classificador('LinearSVC-OneVsRest', estimador))
t.treinar_modelos(dados)

In [15]:
# Multinomial Naive Bayes 
estimador = OneVsOneClassifier(MultinomialNB())
t.adicionar_modelo(Classificador('MultinomialNB-OneVsOne', estimador))
t.treinar_modelos(dados)

estimador = OneVsRestClassifier(MultinomialNB())
t.adicionar_modelo(Classificador('MultinomialNB-OneVsRest', estimador))
t.treinar_modelos(dados)

## Resultados

In [16]:
resultado = pd.DataFrame(dados.metricas)
resultado

,LinearSVC-OneVsOne,LinearSVC-OneVsRest,MultinomialNB-OneVsOne,MultinomialNB-OneVsRest
acuracia,0.9850,0.9833,0.9362,0.9364
precisao,0.9812,0.9814,0.9351,0.9366
revocacao,0.9832,0.9783,0.9260,0.9266
f1,0.9821,0.9798,0.9296,0.9307
